# Processing Dataset using PGSC Code

## Preliminaries

Import Dependencies

In [1]:
import pandas as pd
import os, re

# for helper functions
import sys
sys.path.append('../06_helper_functions')
import helpers

Set Province

In [2]:
province = "Marinduque"

Set Start and End Dates

In [4]:
start_date = pd.to_datetime('2021-01-01')
end_date = pd.to_datetime('2022-12-31')

(start_date, end_date)

(Timestamp('2021-01-01 00:00:00'), Timestamp('2022-12-31 00:00:00'))

## Get the PSGC Codes

Get PSGC Code for Province

In [5]:
prov_psgc = helpers.get_prov_code(province)

prov_psgc

1704000000

Get PSGC Codes for Municipalities

In [8]:
mun_pgsc = helpers.get_mun_codes(int(prov_psgc))
mun_pgsc['adm3_psgc'] = mun_pgsc['adm3_psgc'].astype(object).apply(lambda x: str(x).zfill(10))


#mun_pgsc = mun_pgsc.astype(object).apply(lambda x: str(x).zfill(10))
mun_pgsc

,adm3_en,adm3_psgc
1442,Boac,1704001000
1443,Buenavista,1704002000
1444,Gasan,1704003000
1445,Mogpog,1704004000
1446,Santa Cruz,1704005000
1447,Torrijos,1704006000


## Filter Case Information by Province

Combine all CSV files into one DataFrame

In [9]:
# initialize an empty DataFrame
df_cases = pd.DataFrame()

# get all paths for each CSV file
dir = os.listdir("../01_data/00_raw/00_case_information")

# append each CSV file to DataFrame
for csv_file in dir:
    path = f'../01_data/00_raw/00_case_information/{csv_file}'

    # check if dataframe is empty
    if (df_cases.empty):
        #print(csv_file)
        df_cases = pd.read_csv(path)
    else:
        df_cases = pd.concat([df_cases, pd.read_csv(path)])

C:\Users\diana\AppData\Local\Temp\ipykernel_15992\2709074204.py:14: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cases = pd.read_csv(path)
C:\Users\diana\AppData\Local\Temp\ipykernel_15992\2709074204.py:16: DtypeWarning: Columns (8,11,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cases = pd.concat([df_cases, pd.read_csv(path)])


In [10]:
#verify number of cases
df_cases.shape

(4136488, 23)

In [11]:
df_cases.head()

,CaseCode,Age,AgeGroup,Sex,DateSpecimen,DateResultRelease,DateRepConf,DateDied,DateRecover,RemovalType,...,ProvRes,CityMunRes,CityMuniPSGC,BarangayRes,BarangayPSGC,HealthStatus,Quarantined,DateOnset,Pregnanttab,ValidationStatus
0,C404174,38.0,35 to 39,FEMALE,NaN,2020-01-30,2020-01-30,NaN,NaN,RECOVERED,...,NEGROS ORIENTAL,DUMAGUETE CITY (CAPITAL),PH074610000,NaN,NaN,RECOVERED,NO,2020-01-21,NO,"Removal Type is ""Recovered"", but no Recovered ..."
1,C462688,44.0,40 to 44,MALE,NaN,2020-01-30,2020-02-03,2020-02-01,NaN,DIED,...,NEGROS ORIENTAL,DUMAGUETE CITY (CAPITAL),PH074610000,NaN,NaN,DIED,NO,2020-01-18,NaN,NaN
2,C387710,60.0,60 to 64,FEMALE,2020-01-23,2020-01-30,2020-02-05,NaN,2020-01-31,RECOVERED,...,BOHOL,PANGLAO,PH071233000,NaN,NaN,RECOVERED,NO,2020-01-21,NO,Case has Admitting Facility but is not Admitte...
3,C377460,49.0,45 to 49,MALE,NaN,NaN,2020-03-06,NaN,NaN,RECOVERED,...,BATANGAS,SANTO TOMAS,PH041028000,NaN,NaN,RECOVERED,NO,NaN,NaN,Case has Admitting Facility but is not Admitte...
4,C498051,63.0,60 to 64,MALE,2020-03-05,NaN,2020-03-06,2020-03-11,NaN,DIED,...,RIZAL,CAINTA,PH045805000,NaN,NaN,DIED,NO,NaN,NaN,Age or Birthdate is Invalid\nCase has Lab Resu...


Create column with cleaned PSGC

In [12]:
def clean_psgc(x):
    res_list = re.findall('[0-9]+', str(x))

    try:
        res = res_list[0]

        ## add zero after province
        if len(res) == 9:
            res = res[:2] + '0' + res[2:]
        elif len(res) == 10:
            res = res[:3] + '0' + res[3:]

        return res
    except IndexError:
        return None

# clean psgc code for uniformity, since the format in the DOH case data is different from the PSGC shapefile
df_cases['psgc_clean'] = df_cases['CityMuniPSGC'].apply(clean_psgc)

df_cases.head(10)

,CaseCode,Age,AgeGroup,Sex,DateSpecimen,DateResultRelease,DateRepConf,DateDied,DateRecover,RemovalType,...,CityMunRes,CityMuniPSGC,BarangayRes,BarangayPSGC,HealthStatus,Quarantined,DateOnset,Pregnanttab,ValidationStatus,psgc_clean
0,C404174,38.0,35 to 39,FEMALE,NaN,2020-01-30,2020-01-30,NaN,NaN,RECOVERED,...,DUMAGUETE CITY (CAPITAL),PH074610000,NaN,NaN,RECOVERED,NO,2020-01-21,NO,"Removal Type is ""Recovered"", but no Recovered ...",0704610000
1,C462688,44.0,40 to 44,MALE,NaN,2020-01-30,2020-02-03,2020-02-01,NaN,DIED,...,DUMAGUETE CITY (CAPITAL),PH074610000,NaN,NaN,DIED,NO,2020-01-18,NaN,NaN,0704610000
2,C387710,60.0,60 to 64,FEMALE,2020-01-23,2020-01-30,2020-02-05,NaN,2020-01-31,RECOVERED,...,PANGLAO,PH071233000,NaN,NaN,RECOVERED,NO,2020-01-21,NO,Case has Admitting Facility but is not Admitte...,0701233000
3,C377460,49.0,45 to 49,MALE,NaN,NaN,2020-03-06,NaN,NaN,RECOVERED,...,SANTO TOMAS,PH041028000,NaN,NaN,RECOVERED,NO,NaN,NaN,Case has Admitting Facility but is not Admitte...,0401028000
4,C498051,63.0,60 to 64,MALE,2020-03-05,NaN,2020-03-06,2020-03-11,NaN,DIED,...,CAINTA,PH045805000,NaN,NaN,DIED,NO,NaN,NaN,Age or Birthdate is Invalid\nCase has Lab Resu...,0405805000
5,C130591,58.0,55 to 59,FEMALE,2020-03-06,2020-03-07,2020-03-07,2020-03-12,NaN,DIED,...,CAINTA,PH045805000,NaN,NaN,DIED,NO,NaN,NO,Age or Birthdate is Invalid,0405805000
6,C178743,39.0,35 to 39,MALE,2020-03-06,2020-03-08,2020-03-08,NaN,2020-03-21,RECOVERED,...,CITY OF MAKATI,PH137602000,NaN,NaN,RECOVERED,YES,2020-03-03,NaN,Age or Birthdate is Invalid,1307602000
7,C440075,33.0,30 to 34,MALE,2020-03-06,2020-03-08,2020-03-08,NaN,2020-04-05,RECOVERED,...,NaN,NaN,NaN,NaN,RECOVERED,YES,2020-03-01,NaN,Age or Birthdate is Invalid,None
8,C202135,57.0,55 to 59,MALE,2020-03-06,2020-03-08,2020-03-08,NaN,2020-03-23,RECOVERED,...,NaN,NaN,NaN,NaN,RECOVERED,NO,NaN,NaN,Age or Birthdate is Invalid,None
9,C557002,86.0,80+,MALE,2020-03-06,2020-03-08,2020-03-08,2020-03-14,NaN,DIED,...,CITY OF MARIKINA,PH137402000,NaN,NaN,DIED,NO,2020-03-01,NaN,Age or Birthdate is Invalid,1307402000


Filter by PSGC 

In [14]:
df_cases_filtered = df_cases[df_cases['psgc_clean'].isin(mun_pgsc['adm3_psgc'])]

# note: some case data don't have the psgc code so it's less than the total number of cases if using only province
df_cases_filtered.shape

(5517, 24)

In [15]:
df_cases_filtered.head(10)

,CaseCode,Age,AgeGroup,Sex,DateSpecimen,DateResultRelease,DateRepConf,DateDied,DateRecover,RemovalType,...,CityMunRes,CityMuniPSGC,BarangayRes,BarangayPSGC,HealthStatus,Quarantined,DateOnset,Pregnanttab,ValidationStatus,psgc_clean
510,C256635,23.0,20 to 24,FEMALE,NaN,NaN,2020-03-24,NaN,NaN,RECOVERED,...,BOAC (CAPITAL),PH174001000,NaN,NaN,RECOVERED,YES,NaN,NO,"Removal Type is ""Recovered"", but no Recovered ...",1704001000
535,C299196,24.0,20 to 24,MALE,2020-03-15,2020-03-22,2020-03-25,NaN,2020-03-18,RECOVERED,...,TORRIJOS,PH174006000,NaN,NaN,RECOVERED,YES,2020-03-09,NaN,Age or Birthdate is Invalid,1704006000
1805,C248531,84.0,80+,FEMALE,2020-03-24,2020-03-28,2020-03-31,NaN,2020-04-12,RECOVERED,...,TORRIJOS,PH174006000,NaN,NaN,RECOVERED,YES,2020-03-16,NO,Age or Birthdate is Invalid,1704006000
1954,C206953,3.0,0 to 4,MALE,2020-03-22,2020-03-28,2020-03-31,NaN,2020-04-11,RECOVERED,...,MOGPOG,PH174004000,NaN,NaN,RECOVERED,YES,2020-03-21,NaN,Age or Birthdate is Invalid,1704004000
2452,C604376,52.0,50 to 54,FEMALE,2020-03-24,2020-03-30,2020-04-02,NaN,2020-04-12,RECOVERED,...,TORRIJOS,PH174006000,NaN,NaN,RECOVERED,YES,2020-03-23,NO,Age or Birthdate is Invalid,1704006000
30873,C427500,43.0,40 to 44,MALE,2020-05-27,2020-06-20,2020-06-24,NaN,NaN,RECOVERED,...,GASAN,PH174003000,NaN,NaN,RECOVERED,NO,NaN,NaN,"Health Status is ""Recovered"", but no Date Reco...",1704003000
56446,C495935,51.0,50 to 54,MALE,2020-07-05,2020-07-09,2020-07-16,NaN,NaN,RECOVERED,...,TORRIJOS,PH174006000,NaN,NaN,RECOVERED,YES,2020-07-03,NaN,Age or Birthdate is Invalid\nHealth Status is ...,1704006000
63800,C580690,73.0,70 to 74,FEMALE,2020-07-09,2020-07-14,2020-07-19,NaN,2020-07-28,RECOVERED,...,SANTA CRUZ,PH174005000,NaN,NaN,RECOVERED,YES,2020-07-08,NO,Age or Birthdate is Invalid,1704005000
89652,C260284,51.0,50 to 54,MALE,2020-07-20,2020-07-26,2020-08-01,NaN,2020-08-03,RECOVERED,...,BOAC (CAPITAL),PH174001000,NaN,NaN,RECOVERED,YES,NaN,NaN,Age or Birthdate is Invalid,1704001000
104080,C884966,50.0,50 to 54,MALE,NaN,NaN,2020-08-04,NaN,NaN,RECOVERED,...,TORRIJOS,PH174006000,NaN,NaN,RECOVERED,NO,NaN,NaN,"Health Status is ""Recovered"", but no Date Reco...",1704006000


## Aggregate Data

Aggregate Data for Each Municipality

In [16]:
# create initial empty DataFrame
df_aggregated = pd.DataFrame()

def aggregate_data(df, psgc):
    # for each unique dates, count the number of new cases, deaths, and recoveries
    cases = df.groupby("DateRepConf").size()
    cases.index.name = "Date"

    deaths = df[df["RemovalType"] == "DIED"].groupby("DateRepRem").size()
    deaths.index.name = "Date"

    recoveries = df[df["RemovalType"] == "RECOVERED"].groupby("DateRepRem").size()
    recoveries.index.name = "Date"

    # create a new DataFrame for aggregated data
    new_df = pd.DataFrame()

    data = {
        "NewCases" : cases,
        "Deaths" : deaths,
        "Recoveries" : recoveries
    }

    new_df = pd.concat(data, axis = 1)

    # add information
    new_df["PSGC"] = psgc

    new_df["Municipality"] = mun_pgsc[mun_pgsc['adm3_psgc'] == psgc]['adm3_en'].values[0]

    # sort by date
    new_df = new_df.reset_index()
    new_df["Date"] = pd.to_datetime(new_df.Date, format='mixed')

    new_df = new_df.sort_values(by="Date")
    new_df = new_df.fillna(0)

    return new_df


for psgc in mun_pgsc['adm3_psgc']:
    df_mun = aggregate_data(df_cases_filtered[df_cases_filtered["psgc_clean"] == psgc], psgc)
    df_aggregated = pd.concat([df_aggregated, df_mun], ignore_index=True)


df_aggregated.head(10)


,Date,NewCases,Deaths,Recoveries,PSGC,Municipality
0,2020-03-24,1.0,0.0,0.0,1704001000,Boac
1,2020-05-25,0.0,0.0,1.0,1704001000,Boac
2,2020-08-01,1.0,0.0,0.0,1704001000,Boac
3,2020-08-11,2.0,0.0,0.0,1704001000,Boac
4,2020-08-14,0.0,0.0,1.0,1704001000,Boac
5,2020-08-15,1.0,0.0,0.0,1704001000,Boac
6,2020-08-23,0.0,0.0,2.0,1704001000,Boac
7,2020-08-28,1.0,0.0,0.0,1704001000,Boac
8,2020-08-30,0.0,0.0,1.0,1704001000,Boac
9,2020-09-04,2.0,0.0,0.0,1704001000,Boac


## Compute for Cumulative Cases for each Municipality

Compute for change in cases per row

In [17]:
df_aggregated["d_cases"] = df_aggregated["NewCases"] - df_aggregated["Deaths"] - df_aggregated["Recoveries"]

df_aggregated.head(10)

,Date,NewCases,Deaths,Recoveries,PSGC,Municipality,d_cases
0,2020-03-24,1.0,0.0,0.0,1704001000,Boac,1.0
1,2020-05-25,0.0,0.0,1.0,1704001000,Boac,-1.0
2,2020-08-01,1.0,0.0,0.0,1704001000,Boac,1.0
3,2020-08-11,2.0,0.0,0.0,1704001000,Boac,2.0
4,2020-08-14,0.0,0.0,1.0,1704001000,Boac,-1.0
5,2020-08-15,1.0,0.0,0.0,1704001000,Boac,1.0
6,2020-08-23,0.0,0.0,2.0,1704001000,Boac,-2.0
7,2020-08-28,1.0,0.0,0.0,1704001000,Boac,1.0
8,2020-08-30,0.0,0.0,1.0,1704001000,Boac,-1.0
9,2020-09-04,2.0,0.0,0.0,1704001000,Boac,2.0


Generate Date Range Series for Entire Data

In [18]:
# create series of date from min to max
date_range = pd.date_range(start=df_aggregated['Date'].min(), end=df_aggregated['Date'].max()).to_frame(name="Date")

date_range

,Date
2020-03-24,2020-03-24
2020-03-25,2020-03-25
2020-03-26,2020-03-26
2020-03-27,2020-03-27
2020-03-28,2020-03-28
...,...
2023-12-29,2023-12-29
2023-12-30,2023-12-30
2023-12-31,2023-12-31
2024-01-01,2024-01-01


Calculate cumulative cases per municipality

In [20]:
df_cumsum = pd.DataFrame()

def calculate_cumsum(df, psgc):
    # dataframe with all dates included
    df_extended = pd.DataFrame()
    df_extended["Date"] = date_range

    # merge with existing dataframe
    df_extended = pd.merge(df_extended, df, how='outer', on='Date')

    # add additional data
    df_extended['PSGC'] = psgc
    df_extended["Municipality"] = mun_pgsc[mun_pgsc['adm3_psgc'] == psgc]['adm3_en'].values[0]

    df_extended = df_extended.fillna(0)

    # compute cumulative data
    df_extended["n"] = df_extended["d_cases"].cumsum()

    return df_extended

for psgc in mun_pgsc['adm3_psgc']:
    df_mun = calculate_cumsum(df_aggregated[df_aggregated["PSGC"] == psgc], psgc)
    df_cumsum = pd.concat([df_cumsum, df_mun])

df_cumsum.head(10)

,Date,NewCases,Deaths,Recoveries,PSGC,Municipality,d_cases,n
0,2020-03-24,1.0,0.0,0.0,1704001000,Boac,1.0,1.0
1,2020-03-25,0.0,0.0,0.0,1704001000,Boac,0.0,1.0
2,2020-03-26,0.0,0.0,0.0,1704001000,Boac,0.0,1.0
3,2020-03-27,0.0,0.0,0.0,1704001000,Boac,0.0,1.0
4,2020-03-28,0.0,0.0,0.0,1704001000,Boac,0.0,1.0
5,2020-03-29,0.0,0.0,0.0,1704001000,Boac,0.0,1.0
6,2020-03-30,0.0,0.0,0.0,1704001000,Boac,0.0,1.0
7,2020-03-31,0.0,0.0,0.0,1704001000,Boac,0.0,1.0
8,2020-04-01,0.0,0.0,0.0,1704001000,Boac,0.0,1.0
9,2020-04-02,0.0,0.0,0.0,1704001000,Boac,0.0,1.0


Filter Data Frame by Date

In [21]:
df_cases_filtered_date = df_cumsum[(df_cumsum["Date"] >= start_date) & (df_cumsum["Date"] <= end_date)]
df_cases_filtered_date

,Date,NewCases,Deaths,Recoveries,PSGC,Municipality,d_cases,n
283,2021-01-01,6.0,0.0,0.0,1704001000,Boac,6.0,17.0
284,2021-01-02,2.0,0.0,0.0,1704001000,Boac,2.0,19.0
285,2021-01-03,0.0,0.0,4.0,1704001000,Boac,-4.0,15.0
286,2021-01-04,0.0,0.0,0.0,1704001000,Boac,0.0,15.0
287,2021-01-05,0.0,0.0,0.0,1704001000,Boac,0.0,15.0
...,...,...,...,...,...,...,...,...
1008,2022-12-27,0.0,0.0,0.0,1704006000,Torrijos,0.0,6.0
1009,2022-12-28,0.0,0.0,1.0,1704006000,Torrijos,-1.0,5.0
1010,2022-12-29,0.0,0.0,0.0,1704006000,Torrijos,0.0,5.0
1011,2022-12-30,0.0,0.0,0.0,1704006000,Torrijos,0.0,5.0


## Compute Expected Values

Load population data

In [22]:
# load population dataset
df_pop = pd.read_csv(f'../01_data/01_processed/01_population_data/{province}_population.csv')

df_pop.head(10)

,Municipality,PSGC,2021,2022
0,Boac,1704001000,54730,54730
1,Buenavista,1704002000,23988,23988
2,Gasan,1704003000,34828,34828
3,Mogpog,1704004000,34043,34043
4,Santa Cruz,1704005000,56408,56408
5,Torrijos,1704006000,30524,30524


### Calcurate Average Case rate $r$

Where $r$ is the average case rate for an area for the entire period:

In [23]:
r = 0

year_range = range(int(2021), int(2022)+1)

# calculate rate per year
for year in year_range:
    # get sum per year
    r += (df_cumsum[df_cumsum['Date'].dt.year == year]['n']/ df_pop[f'{year}'].sum()).sum()
    
r

0.19498040687187929

### Calculate expected number of cases per municipality

Create new DataFrame for expected number of cases

In [24]:
df_exp = pd.DataFrame(columns=['PSGC', 'exp'])


for mun in df_pop['PSGC'].unique():
    # calculate expected number of cases, E_i = r*N_i
    N_i = 0

    for year in year_range:
        N_i += df_pop[df_pop['PSGC'] == mun][f'{year}']
    
    # expected number of cases is equal to the average number of cases per day per municipality
    E_i = (r * (N_i/ len(year_range))) / len(pd.date_range(start=start_date, end=end_date))

    # add expected value to row
    df_exp.loc[-1] = [mun, E_i.iloc[0]]
    df_exp.index = df_exp.index + 1 
    df_exp = df_exp.sort_index()


df_exp

,PSGC,exp
0,1.704006e+09,8.152852
1,1.704005e+09,15.066376
2,1.704004e+09,9.092764
3,1.704003e+09,9.302435
4,1.704002e+09,6.407110
5,1.704001e+09,14.618189


Combine results with dataset

In [25]:
# clean psgc code for uniformity
df_exp['PSGC_old'] = df_exp['PSGC']
df_exp['PSGC'] = df_exp['PSGC_old'].astype(int).astype(str)
df_exp['PSGC'] = df_exp['PSGC'].apply(lambda x: str(x).zfill(10))

df_exp

,PSGC,exp,PSGC_old
0,1704006000,8.152852,1.704006e+09
1,1704005000,15.066376,1.704005e+09
2,1704004000,9.092764,1.704004e+09
3,1704003000,9.302435,1.704003e+09
4,1704002000,6.407110,1.704002e+09
5,1704001000,14.618189,1.704001e+09


In [26]:
# merge data frames with computed expected values
df_cumsum_exp = pd.merge(df_cases_filtered_date, df_exp, on='PSGC')
df_cumsum_exp

,Date,NewCases,Deaths,Recoveries,PSGC,Municipality,d_cases,n,exp,PSGC_old
0,2021-01-01,6.0,0.0,0.0,1704001000,Boac,6.0,17.0,14.618189,1.704001e+09
1,2021-01-02,2.0,0.0,0.0,1704001000,Boac,2.0,19.0,14.618189,1.704001e+09
2,2021-01-03,0.0,0.0,4.0,1704001000,Boac,-4.0,15.0,14.618189,1.704001e+09
3,2021-01-04,0.0,0.0,0.0,1704001000,Boac,0.0,15.0,14.618189,1.704001e+09
4,2021-01-05,0.0,0.0,0.0,1704001000,Boac,0.0,15.0,14.618189,1.704001e+09
...,...,...,...,...,...,...,...,...,...,...
4375,2022-12-27,0.0,0.0,0.0,1704006000,Torrijos,0.0,6.0,8.152852,1.704006e+09
4376,2022-12-28,0.0,0.0,1.0,1704006000,Torrijos,-1.0,5.0,8.152852,1.704006e+09
4377,2022-12-29,0.0,0.0,0.0,1704006000,Torrijos,0.0,5.0,8.152852,1.704006e+09
4378,2022-12-30,0.0,0.0,0.0,1704006000,Torrijos,0.0,5.0,8.152852,1.704006e+09


## Export Data to CSV file

Reorder Columns

In [29]:
# this is just for organization and for better readability
df_cumsum_exp.drop('PSGC_old', axis=1)
df_cumsum_exp = df_cumsum_exp.iloc[:, [5, 4, 0, 1, 2, 3, 6, 7, 8]]
df_cumsum_exp

,Municipality,PSGC,Date,NewCases,Deaths,Recoveries,d_cases,n,exp
0,Boac,1704001000,2021-01-01,6.0,0.0,0.0,6.0,17.0,14.618189
1,Boac,1704001000,2021-01-02,2.0,0.0,0.0,2.0,19.0,14.618189
2,Boac,1704001000,2021-01-03,0.0,0.0,4.0,-4.0,15.0,14.618189
3,Boac,1704001000,2021-01-04,0.0,0.0,0.0,0.0,15.0,14.618189
4,Boac,1704001000,2021-01-05,0.0,0.0,0.0,0.0,15.0,14.618189
...,...,...,...,...,...,...,...,...,...
4375,Torrijos,1704006000,2022-12-27,0.0,0.0,0.0,0.0,6.0,8.152852
4376,Torrijos,1704006000,2022-12-28,0.0,0.0,1.0,-1.0,5.0,8.152852
4377,Torrijos,1704006000,2022-12-29,0.0,0.0,0.0,0.0,5.0,8.152852
4378,Torrijos,1704006000,2022-12-30,0.0,0.0,0.0,0.0,5.0,8.152852


In [30]:
df_cumsum_exp.to_csv(f"../01_data/01_processed/00_case_data/{province}_case_data_psgc.csv",  index=False)